In [1]:
import numpy as np
import tensorflow as tf
from sklearn import datasets
from sklearn.preprocessing import minmax_scale, LabelBinarizer

In [2]:
ds = datasets.load_iris()
X, y = minmax_scale(ds.data), LabelBinarizer().fit_transform(ds.target)
data = np.concatenate([X, y], axis = 1)